# TODO:

1) Add standart deviation implementation

2) Add comments

3) Maybe we need to redefine createMatrix for distributive cultures

4) Maybe turn Votings in different classes


# In silico Voting Experiments

Implementation of cultures and voting rules

Code uploaded to the github.

In [1]:
import pandas as pd
import sklearn
import math
import numpy as np
import random
random.seed(12)

# 2 Cultures

In [27]:
class Сulture:
    def __init__(self, n, K):
        self.n = n
        self.K = K
        self.M = np.zeros((self.K, self.K))
        self.ctype = None
        self.cctype = None
    
    def createMatrix(self):
        # matrix K x K 
        
        print(self.ctype)
        if(self.ctype == 'Rousseauist_culture' or self.ctype == 'Impartial_culture'):
            for t in range(self.n):
                self.M += self.createMatrixProfile()
        
        elif(self.ctype == 'Distributive_culture'):
            self.createProfiles() # Distributive_culture matrix creation
            for t in range(self.n):
                self.M += self.createMatrixProfile(self.DC_matrix[t])
        
        elif(self.ctype == 'Spatial_culture'):
            pass
        
        return self.M
    
    
    def linearIntoMatrix(self, profile):
        tempM = np.zeros((self.K, self.K))
        for i in range(self.K):
            for j in range(i+1, self.K):
                tempM[profile[i], profile[j]] += 1
        return tempM

# 2.1 Rousseauist cultures

In [3]:
class Rousseauist_culture(Сulture):
    # We assume that 1 > 2 > ... > K
    def setParams(self, alpha, beta):
        # alpha >= 0
        # beta >= 0
        self.alpha = alpha
        self.beta = beta
        self.ctype = "Rousseauist_culture"
        
    def createMatrixProfile(self):
        tempM = np.zeros((self.K, self.K))
        for i in range(self.K):
            for j in range(i+1, self.K):
                if( random.random() < self.getP(i,j)):
                    tempM[i, j] += 1
                else:
                    tempM[j, i] += 1
        return tempM
        
    def getP(self, k1, k2):
        # Truchon and Drissi-Bakhkhat
        # k > k'
        if(k2 <= k1): 
            raise NameError("k >= k'")
            
        s = math.exp(self.alpha + self.beta*(k2-k1))
        return s/(1+s)

In [4]:
r = Rousseauist_culture(5,6)
r.setParams(1,1)
r.getP(4,5)
print(r.createMatrix())

[[0. 4. 5. 5. 5. 5.]
 [1. 0. 5. 5. 5. 5.]
 [0. 0. 0. 5. 5. 5.]
 [0. 0. 0. 0. 5. 4.]
 [0. 0. 0. 0. 0. 4.]
 [0. 0. 0. 1. 1. 0.]]


# 2.2 Impartial culture

In [5]:
class Impartial_culture(Сulture):
    def setParams(self):
        self.ctype = "Impartial_culture"
    
    def createLinearProfile(self):
        # [1,6,4,3,0,5]: 1 > 6 > 4 > 3 > 0 > 5
        profile = list(range(self.K))
        random.shuffle(profile)
        return profile

    def createMatrixProfile(self):
        pr = self.createLinearProfile()
        return self.linearIntoMatrix(pr)
        

In [6]:
i = Impartial_culture(5,6)
i.setParams()
i.createMatrix()

array([[0., 4., 4., 2., 3., 5.],
       [1., 0., 2., 2., 0., 3.],
       [1., 3., 0., 2., 2., 3.],
       [3., 3., 3., 0., 2., 5.],
       [2., 5., 3., 3., 0., 5.],
       [0., 2., 2., 0., 0., 0.]])

# 2.3 Distributive cultures

Everything is fine except for simple Gini

In [7]:
class Distributive_culture(Сulture):
    def setParams(self):
        self.ctype = "Distributive_culture"
    
    def createMatrixProfile(self, c_pr):
        linear_pr = self.sharesIntoLinear(c_pr)
        tempM = self.linearIntoMatrix(linear_pr)
        return tempM
    
    def sharesIntoLinear(self, shares):
        # [0.61, 0.85, 0.42, 0.13]: [1, 0, 2, 3]: 1 > 0 > 2 > 3 
        linear_pr = [i for (v, i) in sorted([(v, i) for (i, v) in enumerate(shares)], reverse=True)]
        return linear_pr
    
    def GiniIndex(self, c_pr: np.array):
        # Done
        growing = sorted(c_pr)
        accumulated = np.cumsum(growing)
        S = 0
        for i in range(len(accumulated)):
            S += ((i+1)/len(accumulated) - accumulated[i])
        S *= 2/len(accumulated)
        return S
    
    def simpleGiniIndex(self, c_pr: np.array):
        # Problem
        n = len(c_pr)
        growing = sorted(c_pr)
        accumulated = np.cumsum(growing)
        S = 0
        for i in range(n):
            if(i==0):
                S += ( (i+0.5) / n - accumulated[i] / 2)
            else:
                S += ( (i+0.5) / n - (accumulated[i]-accumulated[i-1]) / 2)
        S *= 2/n
        return S
        
    def standartDeviation(self, c_pr: np.array):
        # Done
        return np.std(c_pr)

# 2.3.1 Consensual redistributive culture

Done

In [8]:
# Consensual redistributive culture
class Consensual_redistributive_culture(Distributive_culture):
    def setParams(self):
        self.ctype = "Distributive_culture"
        self.cctype = "Consensual_redistributive_culture"
        self.DC_matrix = None
        
    def createProfiles(self):
        Y = []
        for i in range(self.n):
            Y.append(np.random.random((self.K)))
        X = []
        for i in range(self.n):
            X.append([])
            for j in range(self.K):
                X[-1].append(Y[i][j]/sum([Y[t][j] for t in range(self.n)]))
        self.DC_matrix = X
        return X
        

In [9]:
crc = Consensual_redistributive_culture(n = 4, K = 6)
crc.setParams()
crc.createMatrix()
# for el in crc.DC_matrix:
#     print(crc.standartDeviation(el))

array([[0., 2., 2., 2., 2., 3.],
       [2., 0., 2., 3., 2., 3.],
       [2., 2., 0., 1., 2., 2.],
       [2., 1., 3., 0., 2., 2.],
       [2., 2., 2., 2., 0., 3.],
       [1., 1., 2., 2., 1., 0.]])

## Something strange happend with SimpleGini. We didn't obtained exactly the same results.
Problems with simpleGini. SD and normal Gini work just fine.

In [10]:
def SimulateTable1(times = 100):
    vals = [3,5,11,49,99,999]
    gini_avgs = ['Gini index']
    simple_gini_avgs = ['Simple Gini index']
    standart_deviations = ['Standart deviation']

    for K in vals:
        gini_avg = 0
        simple_gini_avg = 0
        standart_deviation = 0
        for t in range(times):
            crc = Consensual_redistributive_culture(K,1)
            crc.setParams()
            rand = np.random.random((K))
            shares = [el/sum(rand) for el in rand]
            gini_avg += crc.GiniIndex(shares)
            simple_gini_avg += crc.simpleGiniIndex(shares)
            standart_deviation += crc.standartDeviation(shares)
        gini_avg /= times
        gini_avgs.append(gini_avg)
        simple_gini_avg /= times
        simple_gini_avgs.append(simple_gini_avg)
        standart_deviation /= times
        standart_deviations.append(standart_deviation)
    df = pd.DataFrame([standart_deviations, simple_gini_avgs, gini_avgs],columns=['n',3,5,11,49,99, 999])
    return df

df = SimulateTable1()
pd.set_option('display.max_columns', 50)  
pd.set_option('display.width', 1000)
pd.set_option('display.precision', 4)
print(df)


                    n       3       5      11      49      99     999
0  Standart deviation  0.1590  0.1024  0.0522  0.0119  0.0058  0.0006
1   Simple Gini index  0.6667  0.8000  0.9091  0.9796  0.9899  0.9990
2          Gini index  0.2481  0.2745  0.3199  0.3326  0.3296  0.3320


# 2.3.2 Inegalitarian distributive cultures

In [18]:
# Inegalitarian distributive cultures
class Inegalitarian_distributive_culture(Distributive_culture):
    def setParams(self, e_min, e_max):
        self.ctype = "Distributive_culture"
        self.cctype = "Inegalitarian_distributive_culture"
        self.e_min = e_min
        self.e_max = e_max
        
        e = []
        for k in range(self.K):
            e.append(random.uniform(self.e_min, self.e_max))
    
    def createProfiles(self):
        X = [[] for i in range(self.n)]
        e = []
        # define concentration parameter for each k:
        for k in range(self.K):
            e.append(random.uniform(self.e_min, self.e_max))
        for k in range(self.K):
            shares = self.defineShares(e[k])
            random.shuffle(shares)
            for i in range(self.n):
                X[i].append(shares[i])
        self.DC_matrix = X
        return X
        
    def defineShares(self, e):
        v = [0] + [ ( (i+1) / self.n ) ** e  for i in range(self.n)]
        u = [v[i+1]-v[i] for i in range(self.n)]
        return u

In [23]:
crc = Inegalitarian_distributive_culture(4,6)
e_min = 2
e_max = 4
crc.setParams(e_min, e_max)
crc.createMatrix()

array([[0., 1., 2., 2., 3., 2.],
       [3., 0., 2., 2., 2., 1.],
       [2., 2., 0., 3., 2., 1.],
       [2., 2., 1., 0., 2., 2.],
       [1., 2., 2., 2., 0., 2.],
       [2., 3., 3., 2., 2., 0.]])

# 2.4 Spatial cultures

In [13]:
class Spatial_culture(Сulture):
    def setDimention(self, d):
        self.d = d
        self.ctype = "Spatial_culture"
        
    def utility(self, w, x):
        return - np.linalg.norm(x - w, ord=2)
        
    def createProfiles():
        pass
    
    

In [26]:
def generateCultureFile(culture: Сulture):
    if culture.ctype == "Rousseauist_culture":
        pass
    elif culture.ctype == "Impartial_culture":
        pass
    elif culture.ctype == "Distributive_culture":
        pass
    elif culture.ctype == "Spatial_culture":
        pass

# Voting types

In [ ]:
class Voting:
    def __init__(self, culture, matrix):
        self.n = culture.n
        self.K = culture.K
        self.M = matrix
        self.T = [[int(el > self.n/2) for el in line] for line in self.M]
    
    def Borda(self):
        bs = [sum(line) for line in self.M]
        return bs

    def Copeland(self):
        cs = [sum(line) for line in self.T]
        return cs
    
    def checkCondorsetWinner(self):
        cs = self.Copeland()
        if(self.K-1 in cs):
#             print(cs)
            return True
        return False
    
    def Plurality(self):
        pass
    
    def Approval(self):
        pass
    
        

### Tables 3 and 4 simulations:

In [ ]:
def Simulate(n, K, alpha, beta, simulation_runs):
    CondorChance = 0
    for i in range(simulation_runs):
        r = Rousseauist_culture(n,K)
        r.setParams(alpha, beta)
        matrix = r.createMatrix()
        # print(matrix)
        v = Voting(r, matrix)
        CondorChance += int(v.checkCondorsetWinner())
    # print(f'Probability for the Condorset winner  {CondorChance/simulation_runs}')
    return CondorChance/simulation_runs

In [ ]:
simulation_runs = 1000

n = 5
K_values = [5, 11]
alpha_values = [0, 0.4, 0.7, 1]
beta_values  = [0, 0.3, 0.5, 0.7, 1]

for K in K_values:
    print(f"\t\t\t \033[1m  n = {n} K = {K}  \033[0m")
#     print('')
    df = pd.DataFrame(columns=['beta = '+str(beta) for beta in beta_values])
    for alpha in alpha_values:
        line = []
        for beta in beta_values:
            val = Simulate(n, K, alpha, beta, simulation_runs)
            line.append(val)
        df.loc[f'alpha = {alpha}'] = line
    print(df)
    print('\n')


In [ ]:
class Voting_old:
    def __init__(self, filepath, method):
        self.df = pd.read_csv(filepath)
        self.df = self.df.drop('Same', axis=1)
        self.alternatives = set(self.df.columns)
        self.method = method

        # All preferences for each alternative
        self.ALTR_dict = {} # was profile_dict
        for cand in self.alternatives:
            self.ALTR_dict[cand] = self.df[cand].values

        # Preferences for each voter
        self.INDV_d = {}
        for itr, row in self.df.iterrows():
            self.INDV_d[itr]=row.to_dict()

    def voting(self):
        if (self.method == 'Plurality'):
            return self.plurality_voting()
        elif (self.method == 'Borda'):
            return self.borda_voting()
        elif (self.method == 'Nanson'):
            return self.nanson_voting()
        elif (self.method == 'STV'):
            return self.stv_voting()
        elif (self.method == 'Copeland'):
            return self.copeland_voting()
        elif (self.method == 'OurMethod'):
            return self.our_voting()


    # Common fuctions:
    def reboot(self):
        # Normalize
        self.df = self.normalize_df(self.df)
        self.alternatives = set(self.df.columns)

        self.ALTR_dict = {}
        for cand in self.alternatives:
            self.ALTR_dict[cand] = self.df[cand].values

        # Preferences for each voter
        self.INDV_d = {}
        for itr, row in self.df.iterrows():
            self.INDV_d[itr]=row.to_dict()

        return self.df

    def normalize_df(self, df):
        new_df = pd.DataFrame(columns=df.columns)
        for itr, row in df.iterrows():
            new_row = {}
            sorted_dict = sorted(row.to_dict().items(), key=operator.itemgetter(1))
            for i, el in enumerate(sorted_dict):
                new_row[el[0]] = i + 1
            new_df = new_df.append(new_row, ignore_index=True)
        return new_df

    def lexicografic_order(self, winners):
        winners.sort()
        return winners[0]

    def get_winners(self, score_dict):
        winners = []
        max_val = max([i for i in score_dict.values()])
        for key in score_dict.keys():
            if(score_dict[key]==max_val):
                winners.append(key)
        return winners

    def get_losers(self, score_dict):
        losers = []
        if(self.method == 'Nanson'):
            Mean = np.mean([score_dict[cand] for cand in score_dict.keys()])
            for cand in self.alternatives:
                if(score_dict[cand]<Mean):
                    self.df = self.df.drop(cand, axis=1)
        elif(self.method == 'STV'):
            min_val = min([i for i in score_dict.values()])
            for key in score_dict.keys():
                if(score_dict[key]==min_val):
                    losers.append(key)
        elif (self.method == 'OurMethod'):
            max_val = max([i for i in score_dict.values()])
            for key in score_dict.keys():
                if (score_dict[key] == max_val):
                    losers.append(key)
        return losers

    def checkCondorset(self, show=False):
        for el1 in self.alternatives:
            Winner_Flag = True
            for el2 in self.alternatives:
                if(el1!=el2):
                    scores = [0, 0]
                    for itr, row in self.df.iterrows():
                        if(self.df[el1][itr]<self.df[el2][itr]):
                            # el1 - winner
                            scores[0] += 1
                        else:
                            scores[1] += 1
                    if(scores[0]>scores[1]):
                        pass
                    elif (scores[0] < scores[1]):
                        condorset_winner = None
                        Winner_Flag = False
                        break
            if(Winner_Flag):
                if(show):
                    print(f'Alternative {el1} is a Condorcet winner.')
                self.df = self.df.drop(el1, axis=1)
                self.df = self.reboot()
                return True
        return False

    def deleteCondorset(self, show=False):

        Cond_Win = self.checkCondorset(show)
        while (Cond_Win):
            Cond_Win = self.checkCondorset(show)

    # Plurality:
    def plurality_score(self, cand):
        return sum([el == 1 for el in self.ALTR_dict[cand]])

    def plurality_voting(self):
        score_dict = {}
        for cand in self.alternatives:
            score_dict[cand] = self.plurality_score(cand)
        winners = self.get_winners(score_dict)
        return self.lexicografic_order(winners), score_dict

    # Borda:
    def pref_into_borda(self, alternative_prefs, m):
        res_list = []
        for pref in alternative_prefs:
            res_list.append(m-pref)
        return res_list

    def borda_score(self, cand, m):
        return sum(self.pref_into_borda(self.ALTR_dict[cand], m))

    def borda_voting(self):
        score_dict = {}
        m = len(self.alternatives)
        for cand in self.alternatives:
            score_dict[cand] = self.borda_score(cand, m)
        winners = self.get_winners(score_dict)
        return self.lexicografic_order(winners), score_dict

    # Nanson
    def nanson_voting(self):

        score_dict = {}
        m = len(self.alternatives)

        for cand in self.alternatives:
            score_dict[cand] = self.borda_score(cand, m)

        winners = self.get_winners(score_dict)
        while(len(winners)!=len(self.alternatives)):
            losers = self.get_losers(score_dict)
            for loser in losers:
                self.df = self.df.drop(loser, axis=1)

            self.df = self.reboot()
            m = len(self.alternatives)

            score_dict = {}
            for cand in self.alternatives:
                score_dict[cand] = self.borda_score(cand, m)

            winners = self.get_winners(score_dict)

        winners = self.get_winners(score_dict)
        return self.lexicografic_order(winners), score_dict

    # STV
    def stv_score(self, cand):
        return sum([el == 1 for el in self.ALTR_dict[cand]])

    def stv_voting(self):
        # Get scores
        score_dict = {}
        for cand in self.alternatives:
            score_dict[cand] = self.stv_score(cand)
        # Check two or more winners
        winners = self.get_winners(score_dict)
        while(len(winners)!=len(self.alternatives)):
            losers = self.get_losers(score_dict)
            for loser in losers:
                self.df = self.df.drop(loser, axis=1)
            # Normalize
            self.df = self.reboot()

            score_dict = {}
            for cand in self.alternatives:
                score_dict[cand] = self.stv_score(cand)

            winners = self.get_winners(score_dict)

        winners = self.get_winners(score_dict)
        return self.lexicografic_order(winners), score_dict

    # Copeland
    def build_dominance_graph(self):
        graph = {}
        reverse_graph = {}
        for cand in self.alternatives:
            graph[cand] = set()
            reverse_graph[cand] = set()

        for el1 in self.alternatives:
            for el2 in self.alternatives:
                if (el1 != el2):
                    scores = [0, 0]
                    for itr, row in self.df.iterrows():
                        if (self.df[el1][itr] < self.df[el2][itr]):
                            # el1 - winner
                            scores[0] += 1
                        else:
                            scores[1] += 1
                    if (scores[0] > scores[1]):
                        graph[el1].add(el2)
                        reverse_graph[el2].add(el1)
                    elif (scores[0] < scores[1]):
                        graph[el2].add(el1)
                        reverse_graph[el1].add(el2)

        return graph, reverse_graph

    def copeland_score(self, cand):
        return len(self.graph[cand])

    def copeland_voting(self):
        self.graph, self.reversed_graph = self.build_dominance_graph()
        score_dict = {}
        for cand in self.alternatives:
            score_dict[cand] = self.copeland_score(cand)
        winners = self.get_winners(score_dict)
        return self.lexicografic_order(winners), score_dict

